In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2024-08-08 12:32:01.334209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 12:32:01.334402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 12:32:01.478467: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [3]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
#submission

In [6]:
root_file_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
studies = os.listdir(root_file_path)
len(studies[10:110])

100

In [9]:
for study in studies[10:110]:
    for dirname, _, filenames in os.walk(root_file_path+'/'+study):
        for filename in filenames:
            print(os.path.join(dirname, filename))

/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/22.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/25.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/39.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/45.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/2532413137/3557372893/11.dcm
/kaggle/input/rsna-2024-lumbar-spine-d

In [ ]:
Test = True
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']):
            for filename in filenames[config['start']:config['end']]:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [ ]:
len(test_dict)

#### Older Version, takes longer

In [ ]:
"""rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        features = []
                        feature_resized = []
                        for file in test_dict[st]:
                            dicom_ds = dcmread(file)
                            img_array = dicom_ds.pixel_array
                            features.append(np.mean(img_array.T, axis=0))
                        for img in features:
                            feature_resized.append(np.resize(img,(128,)))
                        final = np.array(feature_resized)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])"""

#### New Version

In [ ]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        #features = []
                        feature_resized = []
                        for file in test_dict[st]:
                            dicom_ds = dcmread(file)
                            img_array = dicom_ds.pixel_array
                            feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
                        final = np.array(feature_resized)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

In [ ]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped/2)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

In [ ]:
submission.shape